In [11]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob 

In [5]:
start_time = datetime.now()

In [6]:
dir = 'Y:\\delete\\European Chemicals Agency'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [7]:
def handle_popup():
    try:
        driver.find_elements(by=By.XPATH, value='//*[@id="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton"]/span')[0].click()
    except:
        pass
    return None

In [8]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\European Chemicals Agency"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://echa.europa.eu/nl/pbt")
driver.implicitly_wait(5)
handle_popup()
driver.find_element(by=By.XPATH, value='//*[@id="_disslists_WAR_disslistsportlet_exportButtonXLS"]/span').click()
time.sleep(10)

driver.close()

In [19]:
path = 'Y:\\delete\\European Chemicals Agency'
filenames = glob.glob(path + "\*.xlsx")

for f in filenames:
    dataset = pd.read_excel(f,skiprows=3)
dataset = dataset[['CAS no','Status']]
dataset.rename(columns = {'CAS no':'CAS Number'}, inplace = True)
dataset = dataset.loc[dataset['CAS Number'] != '-']
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)

CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Concluded'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.Status.fillna(result.value, inplace=True)
del result['value']
result.head(2)

d:\python3.9\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,CAS Number,Status
0,1222-05-5,Under development
1,115-27-5,Concluded


In [21]:
result['Jurisdiction'] = 'Europe'
result['RegulatoryAgency'] = 'European Chemicals Agency (ECHA)'
result['TypeOfRegulation'] = 'Environmental'
result['Criteria'] = 'Persistance, bioaccumulation, toxicity (PBT)'

result['Description'] = 'Not Listed as PBT'
result.loc[result['Status'].isin(['vPvB','vPvB#not PBT','PBT#vPvB','PBT']), 'Description'] = 'Confirmed severed PBT, Confirmed PBT'
result.loc[result['Status'].isin(['Under development under SEV','postponed','Under development']), 'Description'] = 'Under PBT investigation'
result.loc[result['Status'].isin(['not PBT#not vPvB','not PBT']), 'Description'] = 'Confirmed not to be PBT'


result['HazardClassification'] = 'Moderate'
result.loc[result['Status'].isin(['vPvB','vPvB#not PBT','PBT#vPvB','PBT']), 'HazardClassification'] = 'Very High'
result.loc[result['Status'].isin(['Under development under SEV','postponed','Under development']), 'HazardClassification'] = 'High'
result.loc[result['Status'].isin(['not PBT#not vPvB','not PBT']), 'HazardClassification'] = 'Low'




result['CauseOfClassification'] = 'INI,D'
result.loc[result['Status'].isin(['vPvB','vPvB#not PBT','PBT#vPvB','PBT']), 'CauseOfClassification'] = 'vPvB,vPvB#not PBT,PBT#vPvB,PBT'
result.loc[result['Status'].isin(['Under development under SEV','postponed','Under development']), 'CauseOfClassification'] = 'Under development under SEV,postponed,Under development'
result.loc[result['Status'].isin(['not PBT#not vPvB','not PBT']), 'CauseOfClassification'] = 'not PBT#not vPvB,not PBT'


result['SourceName'] = 'European Chemicals Agency'
result['SourceURL'] = 'https://echa.europa.eu/nl/pbt'
result = result.drop(['Status'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,1222-05-5,Europe,European Chemicals Agency (ECHA),Environmental,"Persistance, bioaccumulation, toxicity (PBT)",Under PBT investigation,High,"Under development under SEV,postponed,Under de...",European Chemicals Agency,https://echa.europa.eu/nl/pbt
1,115-27-5,Europe,European Chemicals Agency (ECHA),Environmental,"Persistance, bioaccumulation, toxicity (PBT)",Not Listed as PBT,Moderate,"INI,D",European Chemicals Agency,https://echa.europa.eu/nl/pbt


In [ ]:
results.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [22]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:39:36.901547
